In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt


In [2]:
from google.cloud import bigquery

In [3]:
client = bigquery.Client()

Using Kaggle's public dataset BigQuery integration.


In [4]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex6 import *
print("Setup Complete")


Using Kaggle's public dataset BigQuery integration.
Setup Complete


In [5]:

dataset_ref = client.dataset("github_repos", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)
licenses_ref = dataset_ref.table("licenses")

licenses_table = client.get_table(licenses_ref)
df=client.list_rows(licenses_table, max_results=5000).to_dataframe()

In [6]:
df.head()

,repo_name,license
0,gffilm/jpWrapper,artistic-2.0
1,gitpan/Crypt-MagicSignatures-Key,artistic-2.0
2,apisnetworks/apnscp-modules,artistic-2.0
3,bcopy/cours-mcpi,artistic-2.0
4,1220jan/Janet-Stark,artistic-2.0


In [7]:
df.shape

(5000, 2)

In [8]:
files_ref = dataset_ref.table("sample_files")
files_table = client.get_table(files_ref)
df_s=client.list_rows(files_table, max_results=5000).to_dataframe()

In [9]:
df_s.head()

,repo_name,ref,path,mode,id,symlink_target
0,git/git,refs/heads/master,RelNotes,40960,62615ffa4e97803da96aefbc798ab50f949a8db7,Documentation/RelNotes/2.10.0.txt
1,np/ling,refs/heads/master,tests/success/plug_compose.t/plug_compose.ll,40960,0c1605e4b447158085656487dc477f7670c4bac1,../../../fixtures/all/plug_compose.ll
2,np/ling,refs/heads/master,fixtures/strict-par-success/parallel_assoc_lef...,40960,b59bff84ec03d12fabd3b51a27ed7e39a180097e,../all/parallel_assoc_left.ll
3,np/ling,refs/heads/master,fixtures/sequence/parallel_assoc_2tensor2_left.ll,40960,f29523e3fb65702d99478e429eac6f801f32152b,../all/parallel_assoc_2tensor2_left.ll
4,np/ling,refs/heads/master,fixtures/success/my_dual.ll,40960,38a3af095088f90dfc956cb990e893909c3ab286,../all/my_dual.ll


In [10]:
df_s.tail()

,repo_name,ref,path,mode,id,symlink_target
4995,JMRI/JMRI,refs/heads/master,web/css/images/icons-18-black.png,33188,ce1b758ad580663f92c36fb0902afb677ded0b11,None
4996,JMRI/JMRI,refs/heads/master,jython/IoT/testTcpPeripheral.py,33188,7b2b19bf070b407587b4c3cd019bfed30843437f,None
4997,JMRI/JMRI,refs/heads/master,help/en/manual/DecoderPro3/images/dp3_roster_l...,33188,e7ce41fcf0e422c68849a100a48323ca79ca711d,None
4998,JMRI/JMRI,refs/heads/master,java/src/jmri/jmrix/can/cbus/swing/eventtable/...,33188,1ccbadc2b3523e78ceabed5cf2461a10ef92b281,None
4999,JMRI/JMRI,refs/heads/master,xml/decoders/QSI_Articulated_Steam.xml,33188,3c128bbd026960e94587a9a9e97d07984e3c9db4,None


In [11]:
df_s.shape

(5000, 6)

In [12]:
df.columns

Index(['repo_name', 'license'], dtype='object')

In [13]:
df_s.columns

Index(['repo_name', 'ref', 'path', 'mode', 'id', 'symlink_target'], dtype='object')

In [39]:
# Query to determine the number of files per license, sorted by number of files
Q= """
        SELECT L.license, COUNT(1) AS number_of_files
        FROM `bigquery-public-data.github_repos.sample_files` AS sf
        INNER JOIN `bigquery-public-data.github_repos.licenses` AS L 
            ON sf.repo_name = L.repo_name
        GROUP BY L.license
        ORDER BY number_of_files DESC
       """

In [40]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)


In [41]:
qu = client.query(Q, job_config=safe_config)

In [44]:
file_count_by_license_df=qu.to_dataframe()

In [45]:
file_count_by_license_df.shape

(15, 2)

In [46]:
file_count_by_license_df.head()

,license,number_of_files
0,mit,20432387
1,gpl-2.0,16866825
2,apache-2.0,7114400
3,gpl-3.0,4936479
4,bsd-3-clause,2926576
